## GeoMapping LA Traffic Collisions using Folium

### By: Venki Ramachandran

#### Originally printed in medium article by: Madhu Ramiah (all credit to her)
https://madhuramiah.medium.com/geographic-plotting-with-python-folium-2f235cc167b7

In this module you’ll be learning how to use folium to draw mapping and plot markers on them, with zoom capabilities. Prepare to be impressed!!!

**Problem Statement**

Plot the collisions that occurred in LA, the dataset is from Kaggle

https://www.kaggle.com/cityofLA/los-angeles-traffic-collision-data


### Step 1 - Load the data

In [ ]:
#import the libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import folium
from folium import plugins
%matplotlib inline

In [ ]:
#read the dataset and check the first five rows
df = pd.read_csv('traffic-collision-data-from-2010-to-present.csv')
df.head(5)

In [ ]:
#Check the shape of the dataframe
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# Look at the location data and see how to extract the lat and lon
df['Location']

In [ ]:
df['Location'][0]

In [ ]:
# Extract longitute by searching for 'longitude': ' and terminating at 'h'
longitude = df['Location'].str.extract("\'longitude\':\s\'(.+)\',\s\'h", expand=True)
longitude = longitude.applymap(float)
latitude = df['Location'].str.extract("\'latitude\':\s\'(.+)\'}", expand=True)
#print(latitude)
latitude = latitude.applymap(float)
longitude = longitude.rename(columns={0: 'lon'})
latitude = latitude.rename(columns={0: 'lat'})
# create   a new df
coords = pd.concat([latitude, longitude], axis=1)
coords['Area Name'] = df['Area Name']
coords

### Data Handling and Cleaning

Data is clean, do don't bother with this step

In [ ]:
location=coords.groupby(['lat','lon','Area Name']).size().reset_index(name='count').sort_values(by='count',ascending=False)
location=location[location['count']>5]
location['color']=location['count'].apply(lambda count:"Black" if count>=400 else
                                         "green" if count>=300 and count<400 else
                                         "Orange" if count>=200 and count<300 else
                                         "darkblue" if count>=150 and count<200 else
                                         "red" if count>=100 and count<150 else
                                         "lightblue" if count>=75 and count<100 else
                                         "brown" if count>=50 and count<75 else
                                         "grey")
location['size']=location['count'].apply(lambda count:12 if count>=400 else
                                         10 if count>=300 and count<400 else
                                         8 if count>=200 and count<300 else
                                         6 if count>=150 and count<200 else
                                         4 if count>=100 and count<150 else
                                         2 if count>=75 and count<100 else
                                         1 if count>=50 and count<75 else
                                         0.1)
location

In [ ]:
m=folium.Map([34.2012,-118.4662],zoom_start=11)
#location=location[0:2000]
for lat,lon,area,color,count,size in zip(location['lat'],location['lon'],location['Area Name'],location['color'],location['count'],location['size']):
     folium.CircleMarker([lat, lon],
                            popup=area,
                            radius=size,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                            fill_color=color,
                           ).add_to(m)
m.save('LA collisions.html')

In [107]:
# heat map
# convert to (n, 2) nd-array format for heatmap
location1=location[0:100]
location_data = location[['lat', 'lon']].to_numpy()

# plot heatmap
m.add_child(plugins.HeatMap(location_data, radius=15))
m.save('heatmap.html')
# creates the html file in the same dir where the notbook is running

### Open the above html in Google Chrome and you can see the below screen shots

<img src="./Heatmap.png">

<img src="./collisions_markers.png">